In [2]:
model_name = "SniffingNetwork"
stage = "Production"

In [3]:
# Code copied from BuildModel_twoOutputNeurons. Maybe merge?
import tensorflow_io as tfio
from tensorflow.python.ops import gen_audio_ops as audio_ops
import tensorflow.experimental.numpy as tnp

def generate_spectrogram(file_path, label):
    # does not scale as sniffing should be the same independent of background level
    audio_tensor = tfio.audio.AudioIOTensor(file_path, dtype=tf.int16)
    audio = tf.cast(audio_tensor[:], tf.float32)
    spectrogram = audio_ops.audio_spectrogram(audio,
                                              window_size=320,
                                              stride=160,
                                              magnitude_squared=True)
    spectrogram = tf.nn.pool(
        input=tf.expand_dims(spectrogram, -1),
        window_shape=[1, 6],
        strides=[1, 6],
        pooling_type='AVG',
        padding='SAME')
    spectrogram = tf.squeeze(spectrogram, axis=0)
    # Not sure whether the log is a good idea...
    spectrogram = tnp.log10(spectrogram + 1e-6)
    return spectrogram, label

def prepare_data(dir, value):
    filePath = os.path.join(dir, "*.wav")
    files = tf.data.Dataset.list_files(filePath)
    values = tf.zeros(len(files)) if value == 0 else tf.ones(len(files))
    data = tf.data.Dataset.zip((files, tf.data.Dataset.from_tensor_slices(values)))
    spectrogramData = data.map(generate_spectrogram)
    return spectrogramData

def batch_prefetch(dataSet):
    dataSet = dataSet.batch(16)
    dataSet = dataSet.prefetch(8)
    return dataSet

def generate_binary_dataset(trueDataDir, falseDataDir):
    trueData = prepare_data(trueDataDir, 1)
    true_number = trueData.cardinality().numpy()
    print(f"Sniffing Datasets: {true_number}")
    falseData = prepare_data(falseDataDir, 0)
    false_number = falseData.cardinality().numpy()
    print(f"Background Datasets: {false_number}")
    if correct_class_imbalance:
        falseData = falseData.shuffle(false_number).take(true_number)
        false_number = falseData.cardinality().numpy()
        print(f"Corrected Background Datasets: {false_number}")
    combinedData = trueData.concatenate(falseData)
    combinedData = combinedData.cache()
    return combinedData.shuffle(buffer_size=combinedData.cardinality().numpy())

def representative_data_gen():
    data = generate_binary_dataset(trueDataDir=sniffingDir, falseDataDir=backgroundDir)
    train = batch_prefetch(data)
    repr_samples, repr_labels = train.as_numpy_iterator().next()
    yield [repr_samples]

In [16]:
import mlflow.keras
import os
import pathlib
import tensorflow as tf

correct_class_imbalance = True
sniffingDir = "data/train/Sniffing"
backgroundDir = "data/train/Background"

model_uri=f"models:/{model_name}/{stage}"

# TODO Don't make a local copy
model = mlflow.keras.load_model(model_uri)
savedModelDir = os.path.join(os.getcwd(), "sniffing_model/1/")
tf.saved_model.save(model, savedModelDir)
converter = tf.lite.TFLiteConverter.from_saved_model(savedModelDir)

# Post Training Quantization
# https://www.tensorflow.org/lite/performance/post_training_quantization
# Integer with float fallback (using default float input/output)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

# Integer only
# doesn't work here, as I have float32 input and want an output 0<= x <= 1
# atomic14_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# atomic14_converter.inference_input_type = tf.int8  # or tf.uint8
# atomic14_converter.inference_output_type = tf.int8

tflite_model = converter.convert()

tflite_model_file = pathlib.Path('./tflite_models/model.tflite')
tflite_model_file.write_bytes(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Joerg\Documents\tmp\SniffingDetector\Detecting\NeuralNetwork\sniffing_model/1/assets


INFO:tensorflow:Assets written to: C:\Users\Joerg\Documents\tmp\SniffingDetector\Detecting\NeuralNetwork\sniffing_model/1/assets


Sniffing Datasets: 251
Background Datasets: 772
Corrected Background Datasets: 251


43504

In [28]:
import numpy as np

interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_index = input_details[0]['index']
output_index = output_details[0]['index']

# load data
testSniffingDir = "data/test/Sniffing"
testBackgroundDir = "data/test/Background"
testData = generate_binary_dataset(trueDataDir=testSniffingDir, falseDataDir=testBackgroundDir)
testData = batch_prefetch(testData)
num_correct = 0
num_total = 0
for i in range(testData.cardinality().numpy()):
    X_test, y_test = testData.as_numpy_iterator().next()
    for j in range(len(X_test)):
        interpreter.set_tensor(input_index, [X_test[j]])
        interpreter.invoke()
        tflite_result = interpreter.get_tensor(output_index)
        print(tflite_result)
        y_pred = np.argmax(tflite_result, axis=-1)
        print(f"Prediction: {y_pred}")
        # TODO here's something really odd going on y_test changes for each j
        print(f"Truth: {y_test}")
        break
        num_correct += np.sum(y_pred == y_test)
        num_total += len(y_test)

accuracy = num_correct / num_total
print('Accuracy: {:.2f}%'.format(accuracy * 100))

Sniffing Datasets: 64
Background Datasets: 192
Corrected Background Datasets: 64
[[0.         0.99609375]]
Prediction: [1]
Truth: [0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1.]
[[0.90234375 0.09765625]]
Prediction: [0]
Truth: [0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1.]
[[0.         0.99609375]]
Prediction: [1]
Truth: [0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1.]
[[0.99609375 0.        ]]
Prediction: [0]
Truth: [1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1.]
[[0.0078125 0.9921875]]
Prediction: [1]
Truth: [1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0.]
[[0.99609375 0.        ]]
Prediction: [0]
Truth: [1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1.]
[[0.625 0.375]]
Prediction: [0]
Truth: [1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1.]
[[0.         0.99609375]]
Prediction: [1]
Truth: [1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0.]


ZeroDivisionError: division by zero

Run xxd -i model.tflite > model_data.cc in git bash
or install
https://sourceforge.net/projects/xxd-for-windows/

In [31]:
!C:\Users\Joerg\Documents\Programs\xxd.exe -i .\tflite_models\model.tflite > .\tflite_models\model_data.cc